### Import data

In [ ]:
include("importData.jl")
include("Hill.jl")
include("plot.jl")

# import G1, G2, and population data
conc_d, popd, g2d, g1d, g2_0d, g1_0d = setup_data("doxorubicin");

### Hill model, residual functions and optimization

In [ ]:
#  [EC50, b_steepness, alpha_min, alpha_max, beta_min, beta_max, tau1_mean, tau1_max, tau2_min, tau2_max, gamma1_max, gamma2_max]
# guess
guess = [[70.56, 0.05, 0.07, 0.008, 0.08, 0.005, 34.286, 6.56, 8.0, 7.0, 0.005, 0.035], 
         [70.8, 0.02, 0.08, 0.008, 0.03, 0.0045, 20.0, 6.0, 20.0, 10.0, 0.003, 0.02],
         [80.9, 0.03, 0.06, 0.007, 0.05, 0.0035, 15.0, 5.0, 30.0, 5.0, 0.002, 0.03],
         [120.1, 0.04, 0.07, 0.005, 0.03, 0.005, 30.0, 3.0, 20.0, 4.0, 0.003, 0.02],
         [150.8, 0.09, 0.06, 0.009, 0.008, 0.043, 38.94, 5.49, 19.9, 3.2, 0.0029, 0.022]]
# max num of steps
num_steps=2e3
best_fit = zeros(5)
params = zeros(12, 5)
for i in 1:5 
    best_fit[i], params[:, i] = optimize_hill(guess[i], conc_d, g1d, g2d, g1_0d, g2_0d, num_steps)
end


In [ ]:
best, arg = findmin(best_fit)
ps = params[:,arg]

### Plot the data with the new set of parameters for DDE

In [ ]:
# easy way to convert the hill parameters to DDE parameters
p = getDDEparams(ps, conc_d)

# i showas the trial number, which could be from 1:control, ..., 8: maximum drug concentraation
p_1 = plotIt(p[:, 1], g1d, g2d, g2_0d, g1_0d, popd, 1, "Hill doxorubicin")
p_2 = plotIt(p[:, 2], g1d, g2d, g2_0d, g1_0d, popd, 2, "")
p_3 = plotIt(p[:, 3], g1d, g2d, g2_0d, g1_0d, popd, 3, "")
p_4 = plotIt(p[:, 4], g1d, g2d, g2_0d, g1_0d, popd, 4, "")
p_5 = plotIt(p[:, 5], g1d, g2d, g2_0d, g1_0d, popd, 5, "")
p_6 = plotIt(p[:, 6], g1d, g2d, g2_0d, g1_0d, popd, 6, "")
p_7 = plotIt(p[:, 7], g1d, g2d, g2_0d, g1_0d, popd, 7, "")
p_8 = plotIt(p[:, 8], g1d, g2d, g2_0d, g1_0d, popd, 8, "")
plot(p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8)
plot!(size = (1000, 1000))

### Plot Hill curve for all of the DDE model parameters

In [ ]:
# plot
p__1 = plot(conc_d, p[1, :], label = "alpha", linewidth = 2, xlabel = "concentration [nM]", ylabel = "drug effect", line = (0.5, 4, :red))
p__2 = plot(conc_d, p[2, :], label = "beta", linewidth = 2, legend =:best, line = (0.5, 4, :red))
p__3 = plot(conc_d, p[3, :], label = "tau1", linewidth = 2, line = (0.5, 4, :red))
p__4 = plot(conc_d, p[4, :], label = "tau2", linewidth = 2, line = (0.5, 4, :red))
p__5 = plot(conc_d, p[5, :], label = "gamma1", linewidth = 2, line = (0.5, 4, :red))
p__6 = plot(conc_d, p[6, :], label = "gamma2", linewidth = 2, line = (0.5, 4, :red))
plot(p__1, p__2, p__3, p__4, p__5, p__6)
plot!(size = (1000, 700))

In [10]:
using Test
# easy way to convert the hill parameters to DDE parameters
a = [2.0, 4.0]
@test all(x -> x>0, a)
dde_params = [[1.0, 2.0], 
              [3.0, 4.0],
              [1.0, 9.0]]
for i in 1:3
    temp = dde_params[i]
#     print(temp)
    @test all(x -> x>0, temp)
end
# println("this")
# dde_params[1, :]
# a